# Milestone 1

### Differtless: Teresa Datta, Anastasia Ershova, Mark Penrod, Will Wang

## Introduction 
Differentiation of functions helps find the maxima, minima, and zero points that are essential for optimization and prediction in scientific problems. Automatic differentiation (AD) is a computational approach that automatically finds the derivatives of a function using a computer program. For a given function, the AD software should be able to compute the derivatives of any order with machine precision. 

Automatic differentiation is different from classical methods such as symbolic differentiation and numerical differentiation, as AD method requires much less computational time and is not affected by numerical instability unlike classical methods.


Differtless is a software package for performing AD, with additional functionality for performing optimization to find stationary points of functions.

## Background
Automatic differentiation is based on the decomposition of differentials using chain rule. Separated by the order of chain rule calculation, AD can be divided into two modes: forward mode and reverse mode. The forward mode calculates derivatives from the inner function to the outer function using chain rule repeatedly, while the reverse mode calculates from outside to inside. 
We can also say the forward mode calculates the product of the function's Jacobian matrix $J$ with the seed vectors specified by input variables, while reverse modes computes the product of the transpose of Jacobian matrix $J^T$ with  seed vectors. In a function with $m$ inputs and $n$ functions, represented by $R^m \rightarrow R^n$, forward mode is more efficient when $n$ >> $m$, while reverse mode is more efficient when $n$ << $m$.

A computational graph is useful in visualizing the propagation of elementary operations that construct the function to be auto-differentiated. From here, it is easy to evaluate the numerical value after each operation, the derivatives of the elementary operations, and numerical values of derivatives with respect to each input variable.

For an example function 

$$
f(x,y)=
\left[
\begin{matrix}
c_1 & c_2
\end{matrix}
\right]
\left[
\begin{matrix}
w_{11} & w_{12} \\
w_{21} & w_{22}
\end{matrix}
\right]
\left[
\begin{matrix}
x  \\
y
\end{matrix}
\right]
$$

The computational graph is 
<img src="comp.png">

The evaluation trace is 
<img src="trace.png">


## How to use Differtless

Our user will import `differtless`. Then when they have a function that they would like to automatically differentiate, they will: 

1. Define the function, using either elementary operations (which we have overloaded- add, subtract, etc) or `differtless` operations for elementary functions (e.g. `differtless.operations.sin`, `differtless.operations.exp`)

2. User will preprocess their function inputs to create `differtless.ad.FuncInput` objects (and define their own seed matrix if they choose to, otherwise will default to the identity matrix)

3. Call the forward function with their pre-defined function and inputs

This will return the value and corresponding derivatives.

For example:

    import numpy as np
    import differtless as dfl

    def my_fav_function(x, y, z):
        return x*y + dfl.operations.exp(z)


    inputs = [1,2,3]
    #for vector values: inputs = [(1, 2), (3, 4), (5, 6)]
    seeds = np.array([[42, 1, 1], [2, 42, 2], [3, 3, 42]])

    proc_inputs = dfl.ad.preprocess(inputs, seeds)
    # e.g. proc_inputs = [FuncInput(1, [42, 1, 1]), FuncInput(2, [2, 42, 2]), FuncInput(3, [3, 3, 42])] 
   
    evaluation, derivatives = dfl.ad.forward(my_fav_function, my_inputs)

    print(evaluation)

    print(derivatives)

    # additional optimization routine

    my_minimum = dfl.ad.minimize(func = my_fav_function, guess = [42, 42, 42])

    print(my_minimum)


## Software Organization

Directory Structure:

differtless/
* README.md
* setup.py
* .gitignore
* .travis.yml
* docs/
    * milestone1.ipynb *(this document)*
    * tutorial.ipynb
* tests/
    * tests.py
* differtless/
    * operations.py
    * ad.py

Functionality: 
* Documentation will provide both docstring style documentation (in addition to docstrings in .py files) and also interactive short tutorials in the form of jupyter notebooks.
* Tests provides for a comprehensive set of testing programs, and is where our Test Suite will live!
* Operations includes elementary functions such as sin and exp (based on `numpy`), adapted for our datatype to yield their evaluation and derivative
* Forward implements the forward mode of automatic differentiation: preprocessing of user inputs, execution, and optimization functionality (`scipy` required for optimization)

Testing: 
The test suite will live in tests/. We will utilize Travis CI as a continuous integration mechanism to ensure our project is constantly building and passing. We will also use CodeCov to ensure our testing suite is testing all parts and branches of the code. 

We will distribute our package via pip (PyPi) and Conda.

We currently are not intending to utilize a framework for this project.

## Implementation


### Dependencies

`numpy` for elementary functions, `scipy` for optimization

### Core Data Structures

Our implementation, through class `FuncInput` takes advantage of a `list` for storing derivative information. The `list` structure allows us to store multiple values with low overhead. A `tuple` could also be used, but the immutability of `tuple`s could cause unforseen issues later on.

### Handling Vector-Valued Functions

The user can input vectors in order to automatically differentiate vector-valued functions by inputing a tuple or list to represent vector values. During the preprocessing step each of the components will be converted to our `FuncInput` data type.

### `operations` module

Here we define elementary functions for which we cannot use operator overloading to return their evaluation and derivative (requires `numpy` as a dependency).

### `ad` module

* `preprocess` function
    * Allows us to deal with scalars and vectors
    * Takes in a `list` or `numpy.array` of `inputs` (1 x N) and an optional matrix `seeds` (N x N)
    * If the user inputs a scalar it will be converted to a 1 x 1 vector
    * `seeds` defaults to None, in which case we will use an N x N identity matrix where `N = len(inputs)`
    * For each value in `inputs` (and if the inputs are vectors, each component in each vector) and row in `seeds` we will instantiate the `FuncInput` object described below
* `FuncInput` class
    * Parameters: `input` (value – `float` or `int`) and `seed` (seed vector – `list` or `numpy.array`) 
    * The class will have two instance variables: `self.val = value` (value) and `self.der_val = seed` (list of derivative values)
    * In this class we will use dunder methods for operator overloading of elementary operations (e.g. `__add__, __sub__, __mul__, __floordiv__, __truediv__, __mod__, __pow__`) to return evaluation and derivative
    * In our `Operations` module we will implement other elementary operations (e.g. sin, exp)
    * In both cases the methods will return a new instance of `FuncInput` with the `evaluation` and `[evaluation of derivatives]` as inputs
* `forward` function
    * Parameters: `func` (user-defined function, can be scalar or vector), `inputs` (output of preprocessing)
    * Executes function
    * Returns evaluation and derivative calculation
* `minimize` function
    * Parameters: `func` (user-defined function), `guess` (initial guess), `scipy.optimize.minimize` `*args`
    * Uses forward functionality to calculate Jacobian matrix and interfaces with `scipy.optimize.minimize`
    * Returns optimization result

## M1 feedback

2/2 Introduction:
Nice job!

1.5/2 Background
Good start to the background.  The flow could have been enhanced by presenting the evaluation trace and a computational graph.

### Things we add: 

For an example function 

$$
f(x,y)=
\left[
\begin{matrix}
c_1 & c_2
\end{matrix}
\right]
\left[
\begin{matrix}
w_{11} & w_{12} \\
w_{21} & w_{22}
\end{matrix}
\right]
\left[
\begin{matrix}
x  \\
y
\end{matrix}
\right]

$$

The computational graph is 
<img src="comp.png">

The evaluation trace is 
<img src="trace.png">
3/3 How to use
It would be nice to include the installation process.

3/3 Software Organization
Nicely done!

4.5/5 Implementation

1. What are the core data structures and why do use pick these data structures?
2.  Your implementation for vector-valued functions is unclear.

14/15